In [1]:
pip install -Uqq fastbook

Note: you may need to restart the kernel to use updated packages.


In case "Add persistent_workers class variable to FakeLoader " error

please downgrade py torch version by using -> `conda install pytorch==1.6.0 torchvision==0.7.0 -c pytorch` command

* https://github.com/fastai/fastai/pull/2878
* https://github.com/lgvaz/faststyle/pull/18


In [1]:
from sklearn.metrics import classification_report

from fastai.tabular.all import *
from fastai.basics import *
from fastai import *

import numpy as np
import pandas as pd
import glob
# torch.set_num_threads(10)

In [2]:
# load the baseline model
#learn.load('TubularLearner.fastAI._stage1')
deployed_path = "../Models/TubularLearner.II._200.retrain"
#deployed_path = "TubularLearner.fastAI._stage2.pth"
learner = load_learner(deployed_path)



In [3]:
learner.model

TabularModel(
  (embeds): ModuleList()
  (emb_drop): Dropout(p=0.4, inplace=False)
  (bn_cont): BatchNorm1d(1900, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layers): Sequential(
    (0): LinBnDrop(
      (0): BatchNorm1d(1900, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (1): Linear(in_features=1900, out_features=1211, bias=False)
      (2): ReLU(inplace=True)
    )
    (1): LinBnDrop(
      (0): BatchNorm1d(1211, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (1): Linear(in_features=1211, out_features=1211, bias=False)
      (2): ReLU(inplace=True)
    )
    (2): LinBnDrop(
      (0): BatchNorm1d(1211, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (1): Linear(in_features=1211, out_features=1211, bias=False)
      (2): ReLU(inplace=True)
    )
    (3): LinBnDrop(
      (0): BatchNorm1d(1211, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (1): Linear(in_features=1211, o

In [9]:
def toPredictDF(path,to_dir,existing_result):
    appended_pkl = []
    for infile in glob.glob(path):
        #print("Read:",infile)
        file_name = os.path.basename(infile)
        result_path=to_dir+"/"+file_name.replace("pkl", "ML.pkl")
        if result_path in existing_result :
        # print("found then skip : " , result)
            continue
        else:
            df = pd.read_pickle(infile)
            ready_df =df[[ "reps"]]
            df_new = ready_df.reps.apply(pd.Series).astype(np.float64)
            df_new.columns = df_new.columns.astype(str)
            dl = learner.dls.test_dl(df_new)
            _preds,_none ,_y = learner.get_preds(dl=dl, with_decoded=True)
            df.drop(columns=['reps'],inplace =True)
            df['class'] = _y
            #print("Save:",result_path)
            df.to_pickle(result_path)
    print("Complete")

def mergeDF(path,to_dir,file_name):
    appended_data = []
    for infile in glob.glob(path):
        #print(infile)
        data = pd.read_pickle(infile)
        # store DataFrame in list
        appended_data.append(data)
    result_path=to_dir+"/"+file_name
    print("Save:",result_path)
    appended_data = pd.concat(appended_data)
    appended_data.sort_values(by=['ID'], inplace=True)
    appended_data.to_pickle(result_path)
    return appended_data
 

## Poison Frog

In [3]:
existing_result = []
for infile in glob.glob("/mnt/vdb/PoisonFrog/cd100/results/*.pkl"):
    # print(infile)
    existing_result.append(infile)
print(len(existing_result))

0


In [ ]:
# Predict
toPredictDF("/mnt/vdb/PoisonFrog/cd100/reps/*.pkl","/mnt/vdb/PoisonFrog/cd100/results",existing_result)

In [ ]:
# write result
result_df =mergeDF("/mnt/vdb/PoisonFrog/cd100/results/*.pkl","/mnt/vdb/PoisonFrog/cd100/results","PoisonFrog.len15.MLResult.plk")
result_df

In [10]:
result_df

,ID,class
0,0,1
1,8,1
2,10,1
3,15,1
4,18,0
...,...,...
285,12753702,1
286,12753704,1
287,12753708,1
288,12753717,0


## Rat

In [30]:
existing_result = []
for infile in glob.glob("/mnt/vdb/Rat/cd100/result/*.pkl"):
    # print(infile)
    existing_result.append(infile)
print(len(existing_result))

0


In [ ]:
# Predict
toPredictDF("/mnt/vdb/Rat/cd100/reps/*.pkl","/mnt/vdb/Rat/cd100/result",existing_result)

In [33]:
# write result
result_df =mergeDF("/mnt/vdb/Rat/cd100/result/*.pkl","/mnt/vdb/Rat/cd100/result","Rat.len15.MLResult.plk")
result_df

Save: /mnt/vdb/Rat/cd100/result/Rat.len15.MLResult.plk


,ID,class
0,0,1
1,1,0
2,2,0
3,3,0
4,5,0
...,...,...
1049,311701,0
1050,311702,0
1051,311703,0
1052,311705,0


## BAT

In [15]:
existing_result = []
for infile in glob.glob("/mnt/vdb/Bat/cd100/result/*.pkl"):
    # print(infile)
    existing_result.append(infile)
print(len(existing_result))

2514


In [16]:
# Predict
toPredictDF("/mnt/vdb/Bat/cd100/reps/*.pkl","/mnt/vdb/Bat/cd100/result_2_FastAI",existing_result)

Complete


In [17]:
# write result
result_df =mergeDF("/mnt/vdb/Bat/cd100/result_2_FastAI/*.pkl","/mnt/vdb/Bat/cd100/result_2_FastAI","Bat.len10.MLResult.plk")
result_df

Save: /mnt/vdb/Bat/cd100/result_2_FastAI/Bat.len10.MLResult.plk


,ID,length,class
0,9,275,1
1,19,47,0
2,32,151,1
3,54,199,1
4,61,152,1
...,...,...,...
74,12568817,51,1
75,12568833,45,1
76,12568849,117,0
77,12568850,174,1


## Red King

In [5]:
existing_result = []
for infile in glob.glob("/mnt/vdb/RedKing/cd100/result/*.pkl"):
    # print(infile)
    existing_result.append(infile)
print(len(existing_result))

0


In [ ]:
# Predict
toPredictDF("/mnt/vdb/RedKing/cd100/reps/*.pkl","/mnt/vdb/RedKing/cd100/result",existing_result)

In [9]:
# write result
result_df =mergeDF("/mnt/vdb/RedKing/cd100/result/*.pkl","/mnt/vdb/RedKing/cd100/result","RedKing.len10.MLResult.plk")
result_df

Save: /mnt/vdb/RedKing/cd100/result/RedKing.len10.MLResult.plk


,ID,class
0,0,1
1,1,1
2,3,1
3,4,1
4,6,1
...,...,...
25,6492107,1
26,6492113,1
27,6492114,1
28,6492121,1


## Taiwanese Oolong tea

In [37]:
existing_result = []
for infile in glob.glob("/mnt/vdb/Taiwan/cd100/result/*.pkl"):
    # print(infile)
    existing_result.append(infile)
print(len(existing_result))

0


In [ ]:
# Predict
toPredictDF("/mnt/vdb/Taiwan/cd100/reps/*.pkl","/mnt/vdb/Taiwan/cd100/result",existing_result)

In [39]:
# write result
result_df =mergeDF("/mnt/vdb/Taiwan/cd100/result/*.pkl","/mnt/vdb/Taiwan/cd100/result","Taiwan.len20.MLResult.plk")
result_df

Save: /mnt/vdb/Taiwan/cd100/result/Taiwan.len20.MLResult.plk


,ID,class
0,0,1
1,1,1
2,2,1
3,3,0
4,4,1
...,...,...
107,1058123,0
108,1058124,1
109,1058125,1
110,1058126,0


## komodo

In [10]:
existing_result = []
for infile in glob.glob("/mnt/vdb/komodo/cd100/result/*.pkl"):
    # print(infile)
    existing_result.append(infile)
print(len(existing_result))

17248


In [11]:
# Predict
existing_result = []
toPredictDF("/mnt/vdb/komodo/cd100/reps/*.pkl","/mnt/vdb/komodo/cd100/result",existing_result)

Complete


In [18]:
# write result
result_df =mergeDF("/mnt/vdb/komodo/cd100/result/*.pkl","/mnt/vdb/komodo/cd100/result","komodo.len10.MLResult.plk")
result_df

Save: /mnt/vdb/komodo/cd100/result/komodo.len10.MLResult.plk


,ID,class
0,34,1
1,38,0
2,59,0
3,61,1
4,69,1
...,...,...
33,34495360,1
34,34495366,1
35,34495367,1
36,34495419,1


## German cockroach

In [5]:
existing_result = []
for infile in glob.glob("/mnt/vdb/DECockroach/cd100/result_2_FastAI/*.pkl"):
    # print(infile)
    existing_result.append(infile)
print(len(existing_result))

0


In [12]:
# Predict
existing_result = []
toPredictDF("/mnt/vdb/DECockroach/cd100/reps/*.pkl","/mnt/vdb/DECockroach/cd100/result_2_FastAI",existing_result)

Complete


In [13]:
# write result
result_df =mergeDF("/mnt/vdb/DECockroach/cd100/result_2_FastAI/*.pkl","/mnt/vdb/DECockroach/cd100/result_2_FastAI","DECockroach.len15.MLResult.plk")
result_df

Save: /mnt/vdb/DECockroach/cd100/result_2_FastAI/DECockroach.len15.MLResult.plk


,ID,length,class
0,0,17,0
1,1,17,1
2,3,18,0
3,4,30,0
4,5,17,1
...,...,...,...
199,894298,17,1
200,894299,19,0
201,894303,17,1
202,894304,17,0


In [14]:
result_df[result_df["class"]==1]

,ID,length,class
1,1,17,1
4,5,17,1
6,7,18,1
9,10,18,1
10,12,20,1
...,...,...,...
191,894285,33,1
193,894288,17,1
195,894293,28,1
199,894298,17,1
